In [33]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
# a
# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [34]:
import json 
import os

import pandas as pd

In [35]:
!pip install googletrans==4.0.0-rc1

In [36]:
from googletrans import Translator

In [37]:
# from nltk.corpus import wordnet

In [ ]:
# nltk.download('wordnet')

In [38]:
with open('/kaggle/input/nlp-pilot-dataset/pilot_dataset.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [40]:
# data

In [41]:
entries = []
for item in data.values():
    sentence = item['sentence']
    for annotation in item['annotation']:
        entry = {
            'sentence': sentence,
            'target': annotation['target'],
            'snippet': annotation['snippet'],
            'aspect': annotation['aspect'],
            'sentiment': annotation['sentiment'],
            'reasoning': annotation['reasoning']
        }
        entries.append(entry)

# Step 3: Create a DataFrame
df = pd.DataFrame(entries)

In [42]:
df

,sentence,target,snippet,aspect,sentiment,reasoning
0,ICICI Direct is bullish on Tata Motors has rec...,Tata Motors,ICICI Direct is bullish on Tata Motors has rec...,Market/Stock Recommendation,positive,The recommendation to buy indicates a positive...
1,ICICI Direct is bullish on Tata Motors has rec...,Tata Motors,target price of Rs 600,Market/Stock Target Price,positive,A target price of Rs 600 suggests an expected ...
2,"All three verticals of jewellery, watches, and...",jewellery,jewellery reported robust year on year and seq...,Corporate/Operational Health,positive,The robust growth in jewellery indicates stron...
3,"All three verticals of jewellery, watches, and...",watches,watches reported robust year on year and seque...,Corporate/Operational Health,positive,The robust growth in watches signifies healthy...
4,"All three verticals of jewellery, watches, and...",eyewear,eyewear reported robust year on year and seque...,Corporate/Operational Health,positive,The robust growth in eyewear reflects strong o...
...,...,...,...,...,...,...
427,Motilal Oswal recommended Neutral rating on Ne...,Nestle India,Neutral rating on Nestle India with a target p...,Market/Stock Rating,neutral,The neutral rating suggests that the stock is ...
428,Investors would look at the outlook for limite...,Avenue Therapeutics,decision on Avenue Therapeutics,Market/Operational Efficiency,neutral,The mention of a decision on Avenue Therapeuti...
429,Investors would look at the outlook for limite...,inhaler products,update on inhaler products for the US,Market/Operational Efficiency,neutral,An update on inhaler products indicates a focu...
430,Prabhudas Lilladher is bullish on Bajaj Financ...,Bajaj Finance,Prabhudas Lilladher is bullish on Bajaj Financ...,Market/Stock Recommendation,positive,The recommendation to buy the stock indicates ...


In [43]:
class BackTranslation:
    def __init__(self, source_language='en', intermediate_language='es'):
        self.source_language = source_language
        self.intermediate_language = intermediate_language
        self.translator = Translator()

    def translate(self, text):
        # Translate from source to intermediate language
        translated = self.translator.translate(text, src=self.source_language, dest=self.intermediate_language)
        # Translate back from intermediate language to source language
        back_translated = self.translator.translate(translated.text, src=self.intermediate_language, dest=self.source_language)
        return back_translated.text

In [44]:
# Create instances with different intermediate languages: Hindi, Chinese, and Korean
back_translation_hi = BackTranslation(intermediate_language='hi')  # Hindi
back_translation_zh = BackTranslation(intermediate_language='zh-cn')  # Chinese (Simplified)
back_translation_ko = BackTranslation(intermediate_language='ko')  # Korean

# Test the instances
text = "Prabhudas Lilladher is bullish on Mahindra and Mahindra and has recommended a buy rating on the stock."

print("Original:", text)
print("Backtranslation (Hindi):", back_translation_hi.translate(text))
print("Backtranslation (Chinese):", back_translation_zh.translate(text))
print("Backtranslation (Korean):", back_translation_ko.translate(text))

Original: Prabhudas Lilladher is bullish on Mahindra and Mahindra and has recommended a buy rating on the stock.
Backtranslation (Hindi): Prabhudas Lilader is fast on Mahindra and Mahindra and recommended a purchase rating on stock.
Backtranslation (Chinese): Prabhudas Lilladher looks at Mahindra and Mahindra, and recommends buying the stock rating.
Backtranslation (Korean): Prabhudas Lilladher was optimistic at Mahindra and Mahindra and recommended a purchase rating for stocks.


In [46]:
# Function to generate new rows with translated text
def augment_dataframe(df):
    augmented_data = []

    for _, row in df.iterrows():

        original_sentence = row['sentence']

        # Apply back translation for each intermediate language
        translated_hi = back_translation_hi.translate(original_sentence)
        translated_zh = back_translation_zh.translate(original_sentence)
        translated_ko = back_translation_ko.translate(original_sentence)

        # Create 3 new rows for each intermediate language
        augmented_data.append({
            'sentence': original_sentence,
            'target': row['target'],
            'snippet': row['snippet'],
            'aspect': row['aspect'],
            'sentiment': row['sentiment'],
            'reasoning': row['reasoning'],
            'language': 'original'  # Original
        })
        
        augmented_data.append({
            'sentence': translated_hi,
            'target': row['target'],
            'snippet': row['snippet'],
            'aspect': row['aspect'],
            'sentiment': row['sentiment'],
            'reasoning': row['reasoning'],
            'language': 'hi'  # Hindi
        })
        augmented_data.append({
            'sentence': translated_zh,
            'target': row['target'],
            'snippet': row['snippet'],
            'aspect': row['aspect'],
            'sentiment': row['sentiment'],
            'reasoning': row['reasoning'],
            'language': 'zh-cn'  # Chinese
        })
        augmented_data.append({
            'sentence': translated_ko,
            'target': row['target'],
            'snippet': row['snippet'],
            'aspect': row['aspect'],
            'sentiment': row['sentiment'],
            'reasoning': row['reasoning'],
            'language': 'ko'  # Korean
        })

    # Create a new DataFrame with augmented data
    augmented_df = pd.DataFrame(augmented_data)
    return augmented_df



In [50]:
# Example: Assuming `df` is your DataFrame
sampled_df = df.sample(frac=0.15, random_state=42)
sampled_df

,sentence,target,snippet,aspect,sentiment,reasoning
424,Axis Securities is bullish on Britannia Indust...,Britannia Industries,target price of Rs 5110,Market/Stock Target Price,positive,A target price of Rs 5110 suggests an expected...
75,Net Sales are expected to decrease by 6.9 perc...,Net Sales,Net Sales are expected to increase by 1.2 perc...,Financial/Revenue Growth,positive,A quarter-on-quarter increase in net sales sug...
180,Prabhudas Lilladher is bullish on Tech Mahindr...,Tech Mahindra,Prabhudas Lilladher is bullish on Tech Mahindr...,Market/Stock Recommendation,positive,The recommendation to buy indicates a positive...
30,KR Choksey is bullish on SBI Life Insurance Co...,SBI Life Insurance Company,target price of Rs 1705,Market/Stock Target Price,positive,The target price of Rs 1705 suggests an expect...
392,ICICI Direct is bullish on Dr Reddy's Laborato...,Dr Reddy's Laboratories,ICICI Direct is bullish on Dr Reddy's Laborato...,Market/Stock Recommendation,positive,The 'buy' rating indicates a positive outlook ...
...,...,...,...,...,...,...
355,The increase of 32.4 percent in gross revenue ...,consumer businesses,continued growth momentum across consumer busi...,Corporate/Operational Health,positive,The continued growth momentum in consumer busi...
284,Kotak Institutional Equities too expects the b...,the bank,the bank to make higher provisions resulting i...,Corporate/Revenue Base,negative,Higher provisions leading to negligible profit...
79,Motilal Oswal is bullish on Coal India recomme...,Coal India,target price of Rs 325,Market/Stock Target Price,positive,The target price of Rs 325 suggests an anticip...
90,Prabhudas Lilladher is bullish on HDFC Bank ha...,HDFC Bank,target price of Rs 2000,Market/Stock Target Price,positive,A target price of Rs 2000 suggests an expected...


In [52]:
# Generate the augmented DataFrame
augmented_df = augment_dataframe(sampled_df)

In [53]:
augmented_df

,sentence,target,snippet,aspect,sentiment,reasoning,language
0,Axis Securities is bullish on Britannia Indust...,Britannia Industries,target price of Rs 5110,Market/Stock Target Price,positive,A target price of Rs 5110 suggests an expected...,original
1,Axis Securities is fast on Britannia Industrie...,Britannia Industries,target price of Rs 5110,Market/Stock Target Price,positive,A target price of Rs 5110 suggests an expected...,hi
2,AXIS Securities's bullishness of British indus...,Britannia Industries,target price of Rs 5110,Market/Stock Target Price,positive,A target price of Rs 5110 suggests an expected...,zh-cn
3,The AXIS Securities recommended the purchase r...,Britannia Industries,target price of Rs 5110,Market/Stock Target Price,positive,A target price of Rs 5110 suggests an expected...,ko
4,Net Sales are expected to decrease by 6.9 perc...,Net Sales,Net Sales are expected to increase by 1.2 perc...,Financial/Revenue Growth,positive,A quarter-on-quarter increase in net sales sug...,original
...,...,...,...,...,...,...,...
255,Prabhudas Lilladher was optimistic from HDFC B...,HDFC Bank,target price of Rs 2000,Market/Stock Target Price,positive,A target price of Rs 2000 suggests an expected...,ko
256,The RBI has asked HDFC Bank to temporarily hal...,HDFC Bank,stop the launch of its digital business-genera...,Corporate/Product Offering,negative,Stopping the launch of digital business-genera...,original
257,The RBI has asked HDFC Bank to temporarily sou...,HDFC Bank,stop the launch of its digital business-genera...,Corporate/Product Offering,negative,Stopping the launch of digital business-genera...,hi
258,Indian Reserve Bank has asked HDFC Bank to tem...,HDFC Bank,stop the launch of its digital business-genera...,Corporate/Product Offering,negative,Stopping the launch of digital business-genera...,zh-cn


In [55]:
augmented_df.to_csv("perturbed.csv")